In [1]:
import pandas as pd

Constants.

In [2]:
ROOT = '.'
DATA_FOLDER = f'{ROOT}/data'
OUT_FOLDER = f'{ROOT}/results'
FILE = f'{DATA_FOLDER}/Supp. File 1.xlsx'

Read input file.

In [3]:
data = pd.read_excel(FILE, sheet_name='Original Dataset BioDescs')

In [4]:
data = pd.concat([data[['InChIKey', 'Name', 'SMILES', 'Inclusion', 'CMAX_uM']],
                  data.iloc[:, 5:].loc[:, data.iloc[:, 5:].apply(pd.Series.nunique) == 176]       
                  ],
                 axis=1)

In [5]:
labels = pd.read_excel(FILE, sheet_name='Original Dataset Labels', usecols=['InChIKey', 'binaryDILI'])

Merge biological descriptors to molecules' label.

In [6]:
tmp = (labels.merge(data, on='InChIKey')
       .query('Inclusion == "Included"')
       .drop(columns=['InChIKey', 'Name', 'SMILES', 'Inclusion', 'CMAX_uM'])
       )

In [7]:
(tmp.groupby('binaryDILI')
 .describe().T
 .reset_index()
 .rename(columns={'level_0': 'feature',
                  'level_1': 'stat',
                  0: 'neg',
                  1: 'pos'})
 .query('stat.isin(["min", "max", "std"])')
 .pivot(index='feature', columns='stat', values=['neg', 'pos'])
 .T.reset_index(drop=True)
 .T
 .rename(columns={0: 'neg_max', 1: 'neg_min', 2: 'neg_std', 3: 'pos_max', 4: 'pos_min', 5: 'pos_std'})
 .assign(neg_interval_in_pos_interval=lambda x: (x.neg_max < x.pos_max) & (x.neg_min > x.pos_min))
 .to_excel(f'{OUT_FOLDER}/DILI_kept_features_in_interval_pos_neg.xlsx')
)